# Lecture Break/Practice 1 Solutions (Random Forest Speed Dater)
## It will not run in this notebook; needs to be copied over to the `day16.ipynb` notebook

In [ ]:
# Solution
# The classifier via cross validation
# And the confusion matrix
n_splits = 10
max_depth = 4

# initialize k fold
skfold = StratifiedKFold(n_splits=n_splits)

# initialize random forest
rf_clf = RandomForestClassifier(max_depth=max_depth, n_estimators=1000)
# be careful with n_estimators here; because this is a decent sized data set (6K rows) any higher number of trees will run slowly

# initialize y_pred, stores predictions of y
y_pred = np.empty_like(y)

for train_idx, test_idx in skfold.split(x, y):
    # get training data
    x_train = x[train_idx, :]
    y_train = y[train_idx]
    
    # get test data    
    x_test = x[test_idx, :]
    
    # fit data
    rf_clf = rf_clf.fit(x_train, y_train)
    
    # estimate on test data
    y_pred[test_idx] = rf_clf.predict(x_test)

y_feat_list = np.array(['No Match', 'Yes Match'])

# build and plot confusion matrix
sns.set(font_scale=1.2)
conf_mat = confusion_matrix(y_true=y, y_pred=y_pred)
conf_mat_disp = ConfusionMatrixDisplay(conf_mat, display_labels=y_feat_list)
conf_mat_disp.plot()
plt.gcf().set_size_inches(7, 7)
plt.grid(False)
plt.suptitle('max_depth=4 decision trees in a random forest');

In [ ]:
get_acc_sens_spec(y, y_pred)

# Is this useful?

It looks like it's pretty useful! Not only do we have (see below) very high accuracy (overall correct prediction proportion) and sensitivity (true positive rate: the proportion of correctly predicted matches) we also have a perfect sensitivity (we have no false positives; everyone we predict wouldn't go on a date, didn't). Keep in mind, however, that since this is a **random** forest, we may not get the exact same results every time we run it, but as long as the results are consistent if we run it a few times, they should be trustworthy. However, be careful if the feature importance varies greatly (see below), as that might indicate 

In [ ]:
plot_feat_import(x_feat_list, rf_clf.feature_importances_, limit=10)

It seems that (generally) how important the speed dater finds certain activities are more important than the partner's ratings of the importance of different qualities, or of the partner's ratings of the speed dater themselves. Gender and whether the two are the same race also seem to be relatively important (they are usually in the top 10, and we have 33 total x features.

# Lecture Break/Practice 2 Solutions (Regression with EV data set)

In [ ]:
x = np.array(df_ev['Acceleration']).reshape(-1, 1)
y = np.array(df_ev['Top Speed'])

reg = LinearRegression()
reg.fit(x, y) 

# get the slope
slope = reg.coef_[0]

# get the intercept
intercept = reg.intercept_

y_pred = reg.predict(x) # same as y_pred = slope * x + intercept

show_fit(x, y, slope, intercept)

The slope can be interpreted as for every additional second it takes an EV to reach 100 km/hr, the Top Speed of the EV decreases by 10.49 km/hr.

In [ ]:
r2_score(y_true=y, y_pred=y_pred)

76.1% of the variability in Top Speed can be explained by Acceleration.

In [ ]:
# check independence
errors = y - y_pred
plt.scatter(x = range(len(y)), y = errors)
plt.xlabel('index')
plt.ylabel('errors');

In [ ]:
# check constant variance
plt.scatter(x = x, y = errors)
plt.xlabel('Acceleration')
plt.ylabel('errors');

In [ ]:
# check normality
stats.probplot(errors, dist="norm", plot=py)
py.show()

Independence and normality look fairly well met, but constant variance does not. In fact, there is some curving to the errors, which suggests there is another feature that impacts Top Speed which we are not accounting for. This should lead us to examine moving to a multiple and/or polynomial regression framework.